In [3]:
import kfp
from kfp import dsl
import kfp.components as components

In [8]:
import urllib3

print(urllib3.Timeout.DEFAULT_TIMEOUT)

In [9]:
from typing import NamedTuple

def download_and_load_data():
    from minio import Minio
    import numpy as np
    import requests
    import urllib3
    
    minio_client = Minio(
        "192.168.1.10:30950",
        access_key="minio",
        secret_key="minio123",
        secure=False,
        http_client=urllib3.ProxyManager(
            "http://192.168.2.193:9000",
            timeout=urllib3.Timeout.DEFAULT_TIMEOUT,
            # cert_reqs="CERT_REQUIRED",
            retries=urllib3.Retry(
                total=5,
                backoff_factor=0.2,
                status_forcelist=[500, 502, 503, 504],
            ),
        ),
    )
    minio_bucket = "mlpipeline"
    
    filename = "toxic_comments.csv"
    try:
        response = minio_client.get_object(minio_bucket, "dataset/"+filename)
    except:
        url = "https://raw.githubusercontent.com/agungfazrulhaq/toxicdetection/main/data/train.csv"
        r = requests.get(url, allow_redirects=True)
        open(filename, 'wb').write(r.content)

        print("Downloaded file "+filename)
        minio_client.fput_object(minio_bucket, "dataset/toxic_comments.csv", "toxic_comments.csv")
        print("Stored downloaded dataset")

def transform_and_split_train() :
    import tensorflow as tf
    from minio import Minio
    import numpy as np
    import pandas as pd
    import requests
    import re
    import os
    
    filename = "toxic_comments.csv"
    def remove_stopwords(sentence):
        """
        Removes a list of stopwords

        Args:
            sentence (string): sentence to remove the stopwords from

        Returns:
            sentence (string): lowercase sentence without the stopwords
        """
        # List of stopwords
        stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

        # Sentence converted to lowercase-only
        sentence = sentence.lower()

        words = sentence.split()
        no_words = [w for w in words if w not in stopwords]
        sentence = " ".join(no_words)

        return sentence

    def remove_symbols(sentence) :
        return re.sub(r'[^\w]', ' ', sentence)
    
    import urllib3
    
    minio_client = Minio(
        "192.168.1.10:30950",
        access_key="minio",
        secret_key="minio123",
        secure=False,
        http_client=urllib3.ProxyManager(
            "http://192.168.1.10:30950",
            timeout=urllib3.Timeout.DEFAULT_TIMEOUT,
            # cert_reqs="CERT_REQUIRED",
            retries=urllib3.Retry(
                total=5,
                backoff_factor=0.2,
                status_forcelist=[500, 502, 503, 504],
            ),
        ),
    )
    minio_bucket = "mlpipeline"
    
    minio_client.fget_object(minio_bucket, "dataset/toxic_comments.csv", "/tmp/toxic_comments.csv")
    df_train = pd.read_csv("/tmp/toxic_comments.csv")
    train = df_train[['comment_text','toxic','severe_toxic','obscene','threat','insult','identity_hate']]
    
    train["text_no_stopwords"] = train["comment_text"].apply(lambda x : remove_stopwords(x))
    train["text_final"] = train["text_no_stopwords"].apply(lambda x : remove_symbols(x))
    
    split_size=0.9
    def train_val_split(data, split) :
        train = []
        train_label = []
        validation = []
        val_label = []
        for ind,val in data.iterrows() :
            if len(train) < len(data)*split :
                train.append(val['text_final'])
                train_label.append(np.array(val[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].values))
            else :
                validation.append(val['text_final'])
                val_label.append(np.array(val[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].values))

        train = np.array(train)
        train_label = np.array(train_label)
        validation = np.array(validation)
        val_label = np.array(val_label)

        return train, train_label, validation, val_label
    
    x_train, y_train, x_val, y_val = train_val_split(train, split_size)
    
    y_train = np.asarray(y_train).astype(np.float32)
    y_val = np.asarray(y_val).astype(np.float32)
    
    np.save("/tmp/x_train.npy", x_train)

    file_size = os.path.getsize('/tmp/x_train.npy')
    print("File Size is :", file_size, "bytes")
    
    minio_client.fput_object(minio_bucket, "commentoxic/x_train", "/tmp/x_train.npy")
    
    np.save("/tmp/y_train.npy", y_train)
    minio_client.fput_object(minio_bucket, "commentoxic/y_train", "/tmp/y_train.npy")
    
    np.save("/tmp/x_val.npy", x_val)
    minio_client.fput_object(minio_bucket, "commentoxic/x_val", "/tmp/x_val.npy")
    
    np.save("/tmp/y_val.npy", y_val)
    minio_client.fput_object(minio_bucket, "commentoxic/y_val", "/tmp/y_val.npy")
    
    print("Train shape :", x_train.shape)
    print("Validation shape :", x_val.shape)

def building_tokenizer(oov_token:str = "<OOV>"):
    
    from minio import Minio
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.utils import to_categorical 
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    import numpy as np
    
    minio_client = Minio(
        "192.168.1.10:30950",
        access_key="minio",
        secret_key="minio123",
        secure=False
    )
    minio_bucket = "mlpipeline"
    
    def fit_tokenizer(train_sentences, oov_token) :
        tokenizer = Tokenizer(oov_token=oov_token)
        tokenizer.fit_on_texts(train_sentences)
    
        return tokenizer
    
    minio_client.fget_object(minio_bucket,"commentoxic/x_train","/tmp/x_train.npy")
    x_train = np.load("/tmp/x_train.npy")
    
    tokenizer = fit_tokenizer(x_train, oov_token)
    
    with open('tmp/tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    minio_client.fput_object(minio_bucket,"commentoxic/tokenizer.pickle", "/tmp/tokenizer.pickle")
    
    
def building_model(n_epochs:int = 10,
                   optimizer:str = "adam",
                   max_len:int = 120) -> NamedTuple('Output', [('mlpipeline_metrics', 'Metrics')]):
    from minio import Minio
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.utils import to_categorical 
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    import numpy as np
    import os
    
    minio_client = Minio(
        "192.168.1.10:30950",
        access_key="minio",
        secret_key="minio123",
        secure=False
    )
    minio_bucket = "mlpipeline"
    
    EMBEDDING_DIM = 16
    PADDING = 'post'
    TRUNCATING = 'post'
    
    word_index = tokenizer.word_index
    NUM_WORDS = len(word_index)
    
    # Build the model
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(NUM_WORDS, EMBEDDING_DIM, input_length=max_len),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
        tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu'),
        tf.keras.layers.MaxPooling1D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu'),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(6, activation='sigmoid')
    ])
    
    # Print the model summary
    print(model.summary())
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    import pickle
    
    minio_client.fget_object(minio_bucket, "commentoxic/tokenizer.pickle", "/tmp/tokenizer.pickle")
    file = open('/tmp/tokenizer.pickle', 'rb')
    tokenizer = pickle.load(file)
    
    minio_client.fget_object(minio_bucket, "commentoxic/x_train", "/tmp/x_train.npy")
    x_train = np.load("/tmp/x_train.npy")
    
    minio_client.fget_object(minio_bucket, "commentoxic/y_train", "/tmp/y_train.npy")
    y_train = np.load("/tmp/y_train.npy")
    
    minio_client.fget_object(minio_bucket, "commentoxic/x_val", "/tmp/x_val.npy")
    x_val = np.load("/tmp/x_val.npy")
    
    minio_client.fget_object(minio_bucket, "commentoxic/y_val", "/tmp/y_val.npy")
    y_val = np.load("/tmp/y_val.npy")
    

    training_sequences = tokenizer.texts_to_sequences(x_train)
    training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=PADDING, truncating=TRUNCATING)

    validation_sequences = tokenizer.texts_to_sequences(x_val)
    validation_padded = pad_sequences(validation_sequences, maxlen=max_len, padding=PADDING, truncating=TRUNCATING)
    
    # Train the model
    history = model.fit(training_padded, y_train, epochs=num_epochs, validation_data=(validation_padded, y_val))
    
    model.save("/tmp/toxicmodel")
    model.save("/tmp/toxic_model.h5")
    
    minio_client.fput_object(minio_object, "commentoxic/toxic_model.h5", "/tmp/toxic_model.h5")
    
    def upload_local_directory_to_minio(local_path, bucket_name, minio_path):
        assert os.path.isdir(local_path)

        for local_file in glob.glob(local_path + '/**'):
            local_file = local_file.replace(os.sep, "/") # Replace \ with / on Windows
            if not os.path.isfile(local_file):
                upload_local_directory_to_minio(
                    local_file, bucket_name, minio_path + "/" + os.path.basename(local_file))
            else:
                remote_path = os.path.join(
                    minio_path, local_file[1 + len(local_path):])
                remote_path = remote_path.replace(
                    os.sep, "/")  # Replace \ with / on Windows
                minio_client.fput_object(bucket_name, remote_path, local_file)
    
    upload_local_directory_to_minio("/tmp/toxicmodel", minio_bucket, "commentoxic/model/toxicmodel/1/")
    
    metrics = {
      'metrics': [{
          'name': 'model_accuracy',
          'numberValue':  float(history.history["accuracy"][-1]),
          'format' : "PERCENTAGE"
        },{
          'name': 'model_loss',
          'numberValue':  float(history.history["loss"][-1]),
          'format' : "PERCENTAGE"
        },{
          'name': 'model_val_accuracy',
          'numberValue':  float(history.history["val_accuracy"][-1]),
          'format' : "PERCENTAGE"
        },{
          'name': 'model_val_loss',
          'numberValue':  float(history.history["val_loss"][-1]),
          'format' : "PERCENTAGE"
        }]}
    
    from collections import namedtuple
    output = namedtuple('output', ['mlpipeline_metrics'])
    return output(json.dumps(metrics))
    
def model_serving():
    from kubernetes import client 
    from kserve import KServeClient
    from kserve import constants
    from kserve import utils
    from kserve import V1beta1InferenceService
    from kserve import V1beta1InferenceServiceSpec
    from kserve import V1beta1PredictorSpec
    from kserve import V1beta1TFServingSpec
    from datetime import datetime
    
    namespace = utils.get_default_target_namespace()

    now = datetime.now()
    v = now.strftime("%Y-%m-%d--%H-%M-%S")
    
    name='toxic-comment-{}'.format(v)
    kserve_version='v1beta1'
    api_version = constants.KSERVE_GROUP + '/' + kserve_version

    isvc = V1beta1InferenceService(api_version=api_version,
                                   kind=constants.KSERVE_KIND,
                                   metadata=client.V1ObjectMeta(
                                       name=name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                                   spec=V1beta1InferenceServiceSpec(
                                   predictor=V1beta1PredictorSpec(
                                       service_account_name="sa-minio-kserve",
                                       tensorflow=(V1beta1TFServingSpec(
                                           storage_uri="s3://mlpipeline/models/toxicmodel/"))))
    )

    KServe = KServeClient()
    KServe.create(isvc)

# comp_get_data = components.create_component_from_func(download_and_load_data,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0")
# comp_split_train_data = components.create_component_from_func(transform_and_split_train,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0")
# comp_build_tokenizer = components.create_component_from_func(building_tokenizer,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0")
# comp_model_building = components.create_component_from_func(building_model,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0")
# comp_model_serving = components.create_component_from_func(model_serving,base_image="public.ecr.aws/j1r0q0g6/notebooks/notebook-servers/jupyter-tensorflow-full:v1.5.0",
#                                                            packages_to_install=['kserve==0.8.0.1'])


@dsl.pipeline(
    name='toxic-comment-pipeline',
    description='Detect toxicity of a comment'
)
def output_test(oov_token,n_epochs,optimizer,max_len):
    step1 = comp_get_data()
    
    step2 = comp_split_train_data()
    step2.after(step1)
    
    step3 = comp_build_tokenizer(oov_token)
    step3.after(step2)
    
    step4 = comp_model_building(n_epochs,optimizer,max_len)
    step4.after(step3)
    
    step5 = comp_model_serving()
    step5.after(step4)

# if __name__ == "__main__":
#     client = kfp.Client()

#     arguments = {
#         "oov_token":"<OOV>",
#         "n_epochs" : 1,
#         "optimizer": "adam",
#         "max_len": 150,
#     }

#     run_directly = 1
    
#     if (run_directly == 1):
#         client.create_run_from_pipeline_func(output_test,arguments=arguments,experiment_name="test")
#     else:
#         kfp.compiler.Compiler().compile(pipeline_func=output_test,package_path='output_test.yaml')
#         client.upload_pipeline_version(pipeline_package_path='output_test.yaml',pipeline_version_name="0.4",pipeline_name="pipeline test",description="just for testing")

    

In [10]:
transform_and_split_train()

File Size is : 2872280128 bytes


S3Error: S3 operation failed; code: InvalidPart, message: One or more of the specified parts could not be found.  The part may not have been uploaded, or the specified entity tag may not match the part's entity tag., resource: /mlpipeline/commentoxic/x_train, request_id: 1745CACBF8162ABC, host_id: 6810e5c8-b975-4ef0-b5bb-49a4f57b0a8c, bucket_name: mlpipeline, object_name: commentoxic/x_train